<a href="https://colab.research.google.com/github/shahnazumer/LCEL-DOCUMENT/blob/main/LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### [LangChain in Detailed](https://python.langchain.com/docs/expression_language/)

# LangChain Expression Language (LCEL)

The LangChain Expression Language (LCEL) acts as a tool that simplifies and distills certain interesting Python concepts into a format. This format enables the creation of a streamlined code layer for building sequences of LangChain components.

# Advantages of LCEL
## The benefits of using LCEL include:

### Asynchronous, Batch, and Streaming Support:
###### Chains crafted in LCEL inherently cater to both synchronous and asynchronous interfaces, as well as batch and streaming capabilities. This flexibility simplifies the process of starting with a synchronous prototype and later transitioning it to an asynchronous streaming interface.

### Fallback in Chains:
###### With LCEL, any chain can readily incorporate fallbacks. This feature streamlines error management and fosters graceful handling.

### Parallel Processing:
###### LCEL-composed chains inherently support parallel processing for all its components. Given that LLM applications frequently incorporate lengthy API calls, the importance of parallelism cannot be understated.

### Seamless LangSmith Trace Integration:
###### An inherent trait of LCEL is the automatic logging of every step in LangSmith, ensuring optimum observability and debuggability.

#LCEL Syntax

In [ ]:
!pip install langchain==0.0.345 openai

In [6]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [15]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

model = ChatOpenAI(temperature = 0.5)
prompt = ChatPromptTemplate.from_template("Give me brief report about {topic}")
output_parser = StrOutputParser()
chain = prompt | model | output_parser

In [12]:
# In typical LangChain we would chain these three componenets together using an LLMChain but
# Using LCEL the format is different, rather than relying on Chains we simple chain together each component using the pipe operator |:

chain = prompt | model | output_parser

chain.invoke({"topic": "Personal Development"})



"Personal development refers to the ongoing process of improving oneself, both personally and professionally, to reach one's full potential. It involves self-awareness, self-reflection, and self-improvement in various aspects of life, such as relationships, career, health, and personal growth.\n\nThe main goal of personal development is to enhance one's skills, knowledge, and mindset to achieve personal and professional success. It involves setting goals, acquiring new skills, adopting positive habits, and overcoming obstacles to become a better version of oneself.\n\nPersonal development can be achieved through various methods, including reading self-help books, attending seminars and workshops, seeking mentorship, practicing self-reflection, and engaging in activities that promote personal growth. It is a lifelong process that requires continuous learning and adaptation to changing circumstances.\n\nBenefits of personal development include increased self-confidence, improved communic

#LangChain Components and the Runnable Protocol
LangChain components implement the "Runnable" protocol. This protocol provides a standard interface for defining custom chains and invoking them in a standardized manner.

## Standard Interface

##### The primary standard interfaces include:

 - stream: Stream back chunks of response.
 - invoke: Invoke chain with input.
 - batch: Invoke chain with a list of inputs.

And there's a corresponding set of asynchronous methods:

 - astream: Asynchronously streams back a chunk of the response.
 - ainvoke: Asynchronously invokes the chain with an input.
 - abatch: Asynchronously invokes the chain with a list of inputs.
 - astream_log: Streams back intermediate steps in addition to the final response.

In [17]:
# batch response

chain.batch([{"topic": "Personal Development"}, {"topic": "Programming"}])

["Personal development refers to the continuous process of improving oneself mentally, emotionally, physically, and spiritually. It involves setting goals, acquiring new skills, and adopting positive habits to enhance one's overall well-being and achieve personal growth.\n\nThe importance of personal development lies in its ability to help individuals discover their true potential, build self-confidence, and lead a fulfilling life. It enables individuals to identify their strengths and weaknesses, and work towards overcoming obstacles and achieving their goals.\n\nPersonal development can be achieved through various means, such as self-reflection, seeking feedback from others, attending workshops or training programs, reading self-help books, and engaging in activities that promote self-improvement. It involves developing skills like time management, communication, problem-solving, and decision-making, which are essential for success in both personal and professional life.\n\nFurthermo

In [24]:
# Streaming responses

for chunk in chain.stream({"topic" : "Personal Development"}):
    print(chunk, end="", flush=True)

Personal development refers to the process of improving oneself through various activities, experiences, and learning opportunities. It involves the development of skills, knowledge, and attitudes that contribute to personal growth and self-improvement.

Personal development encompasses various aspects of an individual's life, including physical, emotional, intellectual, and spiritual well-being. It often involves setting goals, identifying strengths and weaknesses, and taking steps to enhance one's personal and professional life.

The benefits of personal development are numerous. It can increase self-awareness and self-confidence, improve communication and interpersonal skills, enhance problem-solving abilities, and foster a sense of purpose and fulfillment. Personal development also helps individuals adapt to change, manage stress, and build resilience.

There are various approaches to personal development, including formal education, self-help books, workshops, coaching, and mentor

In [26]:
# Async Stream

async for chunk in chain.astream({"topic" : "Personal Development"}):
    print(chunk, end="", flush=True)

Personal development refers to the ongoing process of improving oneself in various aspects of life, including personal growth, self-awareness, and skill enhancement. It is a lifelong journey that involves setting goals, acquiring new knowledge and skills, and developing positive habits and attitudes.

The importance of personal development lies in its ability to empower individuals to reach their full potential and lead a fulfilling life. It enables individuals to identify their strengths and weaknesses, set realistic goals, and take actions to overcome obstacles and achieve success. Personal development also enhances self-confidence, self-esteem, and self-awareness, leading to improved relationships, career prospects, and overall well-being.

There are several key areas of personal development that individuals can focus on, including:

1. Emotional intelligence: Developing emotional intelligence involves understanding and managing one's own emotions, as well as recognizing and empathi

In [30]:
# Async Invoke

chain.ainvoke({"topic" : "Personal Development"})

# Async Batch

await chain.abatch([{"topic": "Personal Development"}, {"topic": "Chocolates"}])

<ipython-input-30-18070f325ea4>:3: RuntimeWarning: coroutine 'RunnableSequence.ainvoke' was never awaited
  chain.ainvoke({"topic" : "Personal Development"})


["Personal development refers to the continuous process of improving oneself in various aspects of life, including personal, professional, emotional, and social development. It involves setting goals, acquiring new skills, enhancing self-awareness, and fostering positive attitudes and behaviors.\n\nThe purpose of personal development is to maximize one's potential, achieve personal growth, and lead a fulfilling and successful life. It is a lifelong journey that requires self-reflection, self-discipline, and a commitment to personal growth.\n\nThere are several key areas of personal development, including:\n\n1. Self-awareness: This involves understanding one's strengths, weaknesses, values, beliefs, and emotions. It helps individuals gain a deeper understanding of themselves and their motivations.\n\n2. Goal setting: Setting clear and achievable goals is an important aspect of personal development. It helps individuals stay focused, motivated, and accountable for their actions.\n\n3. L

With that in mind, we can build a Runnable class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator |.

In [31]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [32]:
# Let's implement this to take the value 3, add 5 (giving 8), and multiply by 2 (giving 16).

def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

In [34]:
# Using __or__ directly we get the correct answer, now let's try using the pipe operator | to chain them together:

# chain the runnable functions together
chain = add_five | multiply_by_two

# invoke the chain
chain(3)  # we should return 16

16

Using either both above method we get the same response


# Runnables

##### When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use runnables

In [66]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from operator import itemgetter

In [67]:


embeddings = OpenAIEmbeddings()

vecstore_a = FAISS.from_texts(
    ["half the info will be here", "Shahnaz' birthday is the 1st October"],
    embedding=embeddings
)
vecstore_b = FAISS.from_texts(
    ["and half here", "Shahnaz was born in Dubai"],
    embedding=embeddings
)

In [69]:
# First let's try passing a question through to a single one of these vecstore objects:

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | model | output_parser


In [71]:
out = chain.invoke("when was Shahnaz born ?")
print(out)

Shahnaz was born on the 1st October.


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was Shahnaz Born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [72]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | model | output_parser

In [73]:
out = chain.invoke("when was shahnaz born?")
print(out)

Shahnaz was born on the 1st October.


Now the RunnableParallel object is allowing us to search with retriever_a and retriever_b in parallel, ie at the same time.

# Runnable Lambdas


##### The RunnableLambda is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the Runnable class we created near the beginning of this notebook.

##### Let's try it out with our earlier add_five and multiply_by_two functions.

In [74]:
from langchain_core.runnables import RunnableLambda

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

In [75]:
chain = add_five | multiply_by_two

Unlike our Runnable abstraction, we cannot run the RunnableLambda chain by calling it directly, instead we must call chain.invoke:

In [76]:
chain.invoke(3)

16

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [77]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | model | output_parser

In [78]:
chain.invoke({"topic": "Artificial Intelligence"})

'One short fact about Artificial Intelligence is that it is a branch of computer science that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence, such as speech recognition, decision-making, and problem-solving.'

The returned text always includes the initial "Here's a short fact about ...\n\n" — let's add a function to split on double newlines "\n\n" and only return the fact itself.

In [79]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

get_fact = RunnableLambda(extract_fact)

chain = prompt | model | output_parser | get_fact

In [81]:
chain.invoke({"topic": "Artificial Intelligence"})

'One short fact about Artificial Intelligence is that it is a branch of computer science that focuses on creating intelligent machines capable of performing tasks that typically require human intelligence.'

In [82]:
chain.invoke({"topic": "Artificial Intelligence"})

'One short fact about Artificial Intelligence is that it refers to the simulation of human intelligence in machines that are programmed to think and learn like humans, enabling them to perform tasks that typically require human intelligence.'

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.